## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN'], axis=1, inplace=True)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [5]:
# I dont know why we were instructed to drop the names column. I have worked with many NPO's and in my experience 
# there is a degree of bias towards certain names applying. I tried to exclude/bin those with the most applications
# which had little success so I binned only those with a single application

names = application_df['NAME'].value_counts()
# top_50 = list(names.head(50).index)
small = list(names[names<2].index)
# for name in top_50:
#     application_df['NAME'] = application_df['NAME'].replace(name,"Top_50")
for name in small:
  application_df['NAME'] = application_df['NAME'].replace(name,"Single")
print(names)




PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64


In [6]:
affiliation_types=application_df['AFFILIATION'].value_counts()
print(affiliation_types)

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64


In [7]:
# I looked at the affiliation types and binning had little to no effect.

#  affiliation_types_to_replace = affiliation_types.index[affiliation_types.values<100]

# # Replace in dataframe
# for aff in affiliation_types_to_replace:
#     application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")

# # Check to make sure binning was successful
# application_df['AFFILIATION'].value_counts()

In [8]:
# Look at APPLICATION_TYPE value counts for binning
# item_counts = df["col1"].value_counts(normalize=True)
application_types=application_df['APPLICATION_TYPE'].value_counts()
print(application_types)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_types.index[application_types.values<200]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
classification_types=application_df['CLASSIFICATION'].value_counts()
print(classification_types)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_gr1 = classification_types[classification_types.values>1].value_counts
print(classification_gr1)

<bound method IndexOpsMixin.value_counts of C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64>


In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_types.index[classification_types.values<100]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype=float)

In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X= application_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=999)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
count_features = X_train_scaled.shape[1]
nodes_layer1 = 400
nodes_layer2 = 200
nodes_layer3 = 200
nodes_layer4 = 200

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_layer1, activation="relu", input_dim=count_features))

# Second hidden layer
from tensorflow.keras.layers import PReLU, Dense
para_relu = PReLU()
nn.add(tf.keras.layers.Dense(units=nodes_layer2,activation=para_relu))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_layer3, activation=para_relu))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_layer4, activation=para_relu))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 400)               337200    
                                                                 
 dense_1 (Dense)             (None, 200)               80400     
                                                                 
 dense_2 (Dense)             (None, 200)               40400     
                                                                 
 dense_3 (Dense)             (None, 200)               40400     
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 498,201
Trainable params: 498,201
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 9s 8ms/step - loss: 0.4654 - accuracy: 0.7806
Epoch 2/50
804/804 [==============================] - 8s 9ms/step - loss: 0.4188 - accuracy: 0.8031
Epoch 3/50
804/804 [==============================] - 7s 8ms/step - loss: 0.4120 - accuracy: 0.8045
Epoch 4/50
804/804 [==============================] - 7s 9ms/step - loss: 0.4084 - accuracy: 0.8070
Epoch 5/50
804/804 [==============================] - 7s 8ms/step - loss: 0.4071 - accuracy: 0.8077
Epoch 6/50
804/804 [==============================] - 8s 10ms/step - loss: 0.4052 - accuracy: 0.8089
Epoch 7/50
804/804 [==============================] - 8s 9ms/step - loss: 0.4050 - accuracy: 0.8096
Epoch 8/50
804/804 [==============================] - 7s 8ms/step - loss: 0.4035 - accuracy: 0.8094
Epoch 9/50
804/804 [==============================] - 8s 9ms/step - loss: 0.4032 - accuracy: 0.8109
Epoch 10/50
804/804 [==============================] - 7s 8ms/step - loss: 0.4024 - accuracy: 0.810

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6981 - accuracy: 0.8014 - 880ms/epoch - 3ms/step
Loss: 0.69805908203125, Accuracy: 0.8013994097709656


In [ ]:
# Export our model to HDF5 file
from google.colab import files
nn.save('AlphabetSoupCharity_opt3.h5')
files.download('AlphabetSoupCharity_opt3.h5')